In [1]:
#PIGS_X ANALYSIS NOTEBOOK
#Patrick M. Donnelly
#University of Washington March 7th, 2019

In [85]:
# import necessary databases and libraries
import pycurl,json,requests,sys,os,math
import pandas as pd
import numpy as np
from scipy import stats
# existing redcap API stipulates StringIO, but pycurl now works with BytesIO
from io import BytesIO

In [3]:
# get api token
# if you need an access token, consult the RedCap API page
home = os.path.expanduser('~')
with open (home+"/Desktop/redcap_apitoken_repo.txt", "r") as myfile:
    token=myfile.read().split()

In [4]:
# say which report you want
# report 33372 is the report in the repository called "pigs_x data"
pigs_x_report = {
    'token': token,
    'content': 'report',
    'format': 'csv',
    'report_id': '33372',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'returnFormat': 'csv'
}

In [5]:
# specify API address
redcap_path = 'https://redcap.iths.org/api/'
# use requests module to pull data from database using api token
# and convert to csv
pull_pigs_x = requests.post(redcap_path, data=pigs_x_report)
pigs_x_report_filename =os.path.join(home+'/Downloads/pigs_x_data.csv')
with open(pigs_x_report_filename, 'w') as report_file:
        report_file.write(pull_pigs_x.text)
pigs_x_data = pd.read_csv(pigs_x_report_filename)

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from scipy.stats import linregress
from matplotlib import cm
#from colorspacious import cspace_converter
from collections import OrderedDict
cmaps = OrderedDict()
plt.rcParams['pdf.fonttype'] = 42
from numpy.polynomial.polynomial import polyfit

In [29]:
pigs_x_word = pd.melt(pigs_x_data, id_vars = ['record_id', 'pigs_group'], value_name = 'acc', var_name = 'list', 
                        value_vars = ['pigs_word1_acc', 'pigs_word2_acc', 'pigs_word3_acc', 'pigs_word4_acc'])

In [30]:
pigs_x_pseudo = pd.melt(pigs_x_data, id_vars = ['record_id', 'pigs_group'], value_name = 'acc', var_name = 'list', 
                        value_vars = ['pigs_pseudo1_acc', 'pigs_pseudo2_acc', 'pigs_pseudo3_acc', 'pigs_pseudo4_acc'])

In [40]:
pigs_x_word = pigs_x_word.sort_values(by = ['record_id', 'list'])
pigs_x_pseudo = pigs_x_pseudo.sort_values(by = ['record_id', 'list'])
pigs_x_word = pigs_x_word.reset_index()
pigs_x_pseudo = pigs_x_pseudo.reset_index()
del pigs_x_word['index']
del pigs_x_pseudo['index']

In [41]:
pigs_x_word['condition'] = np.nan
pigs_x_pseudo['condition'] = np.nan

In [86]:
for ii in range(0, len(pigs_x_word['record_id'])):
        if pigs_x_word['pigs_group'][ii] == 0.0 or pigs_x_word['pigs_group'][ii] == 2.0:
                if pigs_x_word['list'][ii] == 'pigs_word1_acc' or pigs_x_word['list'][ii] == 'pigs_word3_acc':
                    pigs_x_word.loc[ii,'condition'] = 'normal'
                elif pigs_x_word['list'][ii] == 'pigs_word2_acc' or pigs_x_word['list'][ii] == 'pigs_word4_acc':
                    pigs_x_word.loc[ii, 'condition'] = 'pigs'
                else:
                    raise Exception('Error computing condition (bad list, group 0 or 2)')
        elif pigs_x_word['pigs_group'][ii] == 1.0 or pigs_x_word['pigs_group'][ii] == 3.0:
                if pigs_x_word['list'][ii] == 'pigs_word1_acc' or pigs_x_word['list'][ii] == 'pigs_word3_acc':
                    pigs_x_word.loc[ii,'condition'] = 'pigs'
                elif pigs_x_word['list'][ii] == 'pigs_word2_acc' or pigs_x_word['list'][ii] == 'pigs_word4_acc':
                    pigs_x_word.loc[ii, 'condition'] = 'normal'
                else:
                    raise Exception('Error computing condition (bad list, group 1 or 3)')
        elif math.isnan(pigs_x_word['pigs_group'][ii]):
            continue
        else:
            raise Exception('Error computing condition (bad pigs group)')